# Five Stars - Data Collection
Data was obtained from the Yelp Open Dataset available at https://www.kaggle.com/yelp-dataset/yelp-dataset. The data was downloaded in JSON format and then uploaded to a Mongo database in AWS.

In [1]:
from pymongo import MongoClient
import requests
import json
import pandas as pd
import seaborn as sns


In [2]:
#Original database was hosted in MongoDB on an AWS machine. Connection information hidden here for security.
config = {
    'host': '***.***.***.*',
    'username': '***',
    'password': '***',
    'authSource': '***'
}

client = MongoClient(**config)

The Yelp dataset includes 5 datasets which were each uploaded into a separate collection. We are primarily interested in the 'business' and 'review' collections.

In [3]:
#list collections
db = client.yelp
db.list_collection_names()

['checkin', 'review', 'user', 'tip', 'business']

In [5]:
print(f"Number of businesses: {db.business.estimated_document_count()}")
print(f"Number of reviews: {db.review.estimated_document_count()}")

Number of businesses: 209393
Number of reviews: 8021122


We can see the datasets include information on 200K businesses and over 8 million user reviews. The 'business' collection can be uploaded into a single dataframe but the 'review' collection is too large to handle. Instead we will look at a subset and download reviews for a single city, in this case Toronto.

In [132]:
list(db.business.find().limit(10))

[{'_id': ObjectId('5ebbad25a3830443ce731437'),
  'business_id': 'f9NumwFMBDn751xgFiRbNA',
  'name': 'The Range At Lake Norman',
  'address': '10913 Bailey Rd',
  'city': 'Cornelius',
  'state': 'NC',
  'postal_code': '28031',
  'latitude': 35.4627242,
  'longitude': -80.8526119,
  'stars': 3.5,
  'review_count': 36,
  'is_open': 1,
  'attributes': {'BusinessAcceptsCreditCards': 'True',
   'BikeParking': 'True',
   'GoodForKids': 'False',
   'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
   'ByAppointmentOnly': 'False',
   'RestaurantsPriceRange2': '3'},
  'categories': 'Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping',
  'hours': {'Monday': '10:0-18:0',
   'Tuesday': '11:0-20:0',
   'Wednesday': '10:0-18:0',
   'Thursday': '11:0-20:0',
   'Friday': '11:0-20:0',
   'Saturday': '11:0-20:0',
   'Sunday': '13:0-18:0'}},
 {'_id': ObjectId('5ebbad25a3830443ce731438'),
  'business_id': 'Yzvjg0SayhoZgCljUJRF9Q',
  'name': 'Carlos 

In [25]:
#Save 'business' collection to a dataframe
df_business = pd.DataFrame(list(db.business.find({},{'_id':0,'business_id':1, 'city':1, 'state':1, 'postal_code':1, 'stars':1, 'categories':1,'is_open':1, 'name':1, 'review_count':1})))

In [46]:
df_business.to_pickle('data/yelp_business_df.pkl')

In [4]:
df_business = pd.read_pickle('data/yelp_business_df.pkl')
df_business

,business_id,name,city,state,postal_code,stars,review_count,is_open,categories
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,Cornelius,NC,28031,3.5,36,1,"Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD",Scottsdale,AZ,85258,5.0,4,1,"Health & Medical, Fitness & Instruction, Yoga,..."
2,XNoUzKckATkOD1hP6vghZg,Felinus,Montreal,QC,H4C 1P4,5.0,5,1,"Pets, Pet Services, Pet Groomers"
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,North Las Vegas,NV,89030,2.5,3,0,"Hardware Stores, Home Services, Building Suppl..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,Mesa,AZ,85205,4.5,26,1,"Home Services, Plumbing, Electricians, Handyma..."
...,...,...,...,...,...,...,...,...,...
209388,9Q0fPWAjUweoFDk0kafuzQ,Nishi Sushi,Vaughan,ON,L4H 2P2,4.0,5,0,"Japanese, Sushi Bars, Restaurants"
209389,PotiAhtDMYMNsJCzV0oLAw,Walmart,Charlotte,NC,28208,2.0,13,0,"Department Stores, Food, Mobile Phones, Fashio..."
209390,SYa2j1boLF8DcGVOYfHPcA,Five Guys,Scottsdale,AZ,85251,3.5,97,1,"American (New), Food, Burgers, Restaurants, Fa..."
209391,RSSIsgO00OuWQTRoITacpA,Indian Trail Dog Training,Indian Trail,NC,28079,5.0,3,1,"Pet Services, Pet Training, Pets"


In [148]:
df_business.describe()

,stars,review_count,is_open
count,209393.000000,209393.000000,209393.000000
mean,3.538055,36.937505,0.806632
std,1.023543,123.343597,0.394940
min,1.000000,3.000000,0.000000
25%,3.000000,4.000000,1.000000
50%,3.500000,9.000000,1.000000
75%,4.500000,27.000000,1.000000
max,5.000000,10129.000000,1.000000


In [123]:
mask = (df_business['categories'].str.contains('Restaurants') == True) & (df_business.city.isin(['Las Vegas','Phoenix','Charlotte','Pittsburgh','Toronto','Montreal','Calgary']))
df_business[mask].groupby('city').is_open.mean().sort_values()

city
Toronto       0.629335
Las Vegas     0.631655
Phoenix       0.656154
Pittsburgh    0.670629
Charlotte     0.695414
Calgary       0.728264
Montreal      0.756757
Name: is_open, dtype: float64

In [110]:
df_business[mask].groupby(['city']).review_count.sum().sort_values(ascending=False).head(40)

city
Las Vegas            1455115
Phoenix               492298
Toronto               420499
Scottsdale            264074
Charlotte             251373
Pittsburgh            188626
Tempe                 136360
Henderson             134738
Montréal              133332
Mesa                  105991
Chandler              103245
Cleveland              93639
Gilbert                84424
Madison                80807
Calgary                68525
Glendale               67300
Mississauga            48507
Markham                45820
Peoria                 36756
North Las Vegas        29976
Surprise               23441
Richmond Hill          23253
Champaign              23194
Goodyear               18865
Scarborough            16834
North York             16826
Concord                15408
Lakewood               15273
Avondale               15270
Vaughan                14730
Matthews               11717
Huntersville           11506
Brampton               11022
Cave Creek             10178
Etobicoke

In [114]:
df_business[mask].city.value_counts().sort_values(ascending=False).head(40)

Toronto            8679
Las Vegas          6931
Phoenix            4217
Montréal           4097
Calgary            3117
Charlotte          2922
Pittsburgh         2608
Scottsdale         1652
Mississauga        1612
Cleveland          1550
Mesa               1238
Madison            1175
Tempe              1094
Henderson           950
Markham             926
Chandler            920
Glendale            771
Gilbert             616
Brampton            603
Richmond Hill       563
Scarborough         561
Vaughan             534
North York          491
Champaign           443
Peoria              383
Concord             363
North Las Vegas     362
Laval               331
Etobicoke           329
Surprise            247
Matthews            218
Gastonia            213
Newmarket           209
Mentor              198
Avondale            198
Akron               196
Fort Mill           196
Oakville            191
Goodyear            191
Lakewood            189
Name: city, dtype: int64

In [138]:
#Saving business_ids for Toronto restaurants. This will then be used to get subset of 'review' collection.
#This subset contains a more manageable 400K reviews.
ids = db.business.find({'city':'Toronto','categories':{'$regex': 'Restaurants'}}).distinct('business_id')
reviews = db.review.find({'business_id':{'$in': ids}},{'_id':0,'business_id':1, 'stars':1, 'text':1})

In [142]:
reviews.count()

/home/andy/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


432479

In [143]:
df_review = pd.DataFrame(list(reviews))

In [152]:
df_review.to_pickle('data/yelp_review_toronto.pkl')

In [154]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209393 entries, 0 to 209392
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   209393 non-null  object 
 1   name          209393 non-null  object 
 2   city          209393 non-null  object 
 3   state         209393 non-null  object 
 4   postal_code   209393 non-null  object 
 5   stars         209393 non-null  float64
 6   review_count  209393 non-null  int64  
 7   is_open       209393 non-null  int64  
 8   categories    208869 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 14.4+ MB


In [161]:
#Merging 'review' data with 'business' data
df_toronto = pd.merge(df_business.loc[:,['business_id','postal_code','review_count','is_open','stars']], df_review, on='business_id', how='right')
df_toronto = df_toronto.rename(columns={'stars_x':'stars_bus','stars_y':'stars_user'})

In [179]:
#Final pickle file to be used in main analysis
df_toronto.to_pickle('yelp_rest_toronto.pkl')